# TensorFlow Data Validation Example

This notebook describes how to explore and validate Chicago Taxi dataset using TensorFlow Data Validation.

# Setup

Import necessary packages and set up data paths.

In [ ]:
import tensorflow_data_validation as tfdv
import os

In [ ]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data')
TRAIN_DATA = os.path.join(DATA_DIR, 'train.csv')
EVAL_DATA = os.path.join(DATA_DIR, 'eval.csv')
SERVING_DATA = os.path.join(DATA_DIR, 'serving.csv')

# Compute descriptive data statistics

TFDV can compute descriptive
[statistics](https://github.com/tensorflow/metadata/tree/v0.6.0/tensorflow_metadata/proto/v0/statistics.proto)
that provide a quick overview of the data in terms of the features that are
present and the shapes of their value distributions.

Internally, TFDV uses [Apache Beam](https://beam.apache.org)'s data-parallel
processing framework to scale the computation of statistics over large datasets.
For applications that wish to integrate deeper with TFDV (e.g., attach
statistics generation at the end of a data-generation pipeline), the API also
exposes a Beam PTransform for statistics generation.

In [ ]:
train_stats = tfdv.generate_statistics_from_csv(TRAIN_DATA)

The statistics can be visualized using [Facets Overview](https://pair-code.github.io/facets/) tool which provide a succinct visualization of these statistics for easy browsing. TFDV provides a utility method that visualizes statistics using Facets.

In [ ]:
tfdv.visualize_statistics(train_stats)

# Infer a schema

The
[schema](https://github.com/tensorflow/metadata/tree/v0.6.0/tensorflow_metadata/proto/v0/schema.proto)
describes the expected properties of the data. Some of these properties are:

*   which features are expected to be present
*   their type
*   the number of values for a feature in each example
*   the presence of each feature across all examples
*   the expected domains of features.

In short, the schema describes the expectations for "correct" data and can thus
be used to detect errors in the data (described below). 

Since writing a schema can be a tedious task, especially for datasets with lots
of features, TFDV provides a method to generate an initial version of the schema
based on the descriptive statistics.

In [ ]:
schema = tfdv.infer_schema(train_stats)

In general, TFDV uses conservative heuristics to infer stable data properties
from the statistics in order to avoid overfitting the schema to the specific
dataset. It is strongly advised to **review the inferred schema and refine
it as needed**, to capture any domain knowledge about the data that TFDV's
heuristics might have missed.

In [ ]:
tfdv.display_schema(schema)

# Check evaluation data for errors

Given a schema, it is possible to check whether a dataset conforms to the
expectations set in the schema or whether there exist any data anomalies. TFDV
performs this check by matching the statistics of the dataset against the schema
and marking any discrepancies. 

In [ ]:
# Compute stats over eval data.
eval_stats = tfdv.generate_statistics_from_csv(EVAL_DATA)

In [ ]:
# Compare stats of eval data with training data.
tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

In [ ]:
# Check eval data for errors by validating the eval data stats using the previously inferred schema.
anomalies = tfdv.validate_statistics(eval_stats, schema)

In [ ]:
tfdv.display_anomalies(anomalies)

The anomalies indicate that out of domain values were found for features `company` and `payment_type` in the stats in < 1% of the examples. If this was expected, then the schema can be updated as follows.

In [ ]:
# Relax the minimum fraction of values that must come from the domain for feature company.
company = tfdv.get_feature(schema, 'company')
company.distribution_constraints.min_domain_mass = 0.9

# Add new value to the domain of feature payment_type.
payment_type_domain = tfdv.get_domain(schema, 'payment_type')
payment_type_domain.value.append('Prcard')

In [ ]:
updated_anomalies = tfdv.validate_statistics(eval_stats, schema)

In [ ]:
tfdv.display_anomalies(updated_anomalies)

If an anomaly truly indicates a data error, then the underlying data should be fixed.

# Schema Environments

By default, validations assume that all datasets in a pipeline adhere to a single schema. In some cases introducing
slight schema variations is necessary, for instance features used as labels are required during training (and should
be validated), but are missing during serving.

**Environments** can be used to express such requirements. In particular, features in schema can be associated with a set of environments using `default_environment`, `in_environment` and `not_in_environment`.

For example, if the `tips` feature is being used as the label in training, but missing in the serving data. Without environment specified, it will show up as an anomaly.

In [ ]:
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema)

In [ ]:
tfdv.display_anomalies(serving_anomalies)

Note that 'tips' feature is shown in the anomalies as 'Column dropped', as it is not present in the serving dataset. We can do the following to fix this.

In [ ]:
# All features are by default in both TRAINING and SERVING environments.
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

# Specify that 'tips' feature is not in SERVING environment.
tfdv.get_feature(schema, 'tips').not_in_environment.append('SERVING')

serving_anomalies_with_env = tfdv.validate_statistics(
    serving_stats, schema, environment='SERVING')

In [ ]:
tfdv.display_anomalies(serving_anomalies_with_env)

# Check data drift and skew

In addition to checking whether a dataset conforms to the expectations set in the schema, TFDV also provides functionalities to detect
* drift between different days of training data
* skew between training and serving data

TFDV performs this check by comparing the statistics of different datasets based on the drift/skew comparators specified in the schema.

In [ ]:
# Add skew comparator for 'payment_type' feature.
payment_type = tfdv.get_feature(schema, 'payment_type')
payment_type.skew_comparator.infinity_norm.threshold = 0.01

# Add drift comparator for 'company' feature.
company=tfdv.get_feature(schema, 'company')
company.drift_comparator.infinity_norm.threshold = 0.001

In [ ]:
skew_anomalies = tfdv.validate_statistics(train_stats, schema, previous_statistics=eval_stats,
                                          serving_statistics=serving_stats)

In [ ]:
tfdv.display_anomalies(skew_anomalies)